In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# read table

dataPayment = '../Datasets/Employee_Payroll.csv'
payroll = pd.read_csv(dataPayment)

# set default value to 0 for NaN numerical data
numeric_cols = payroll.select_dtypes(include=[np.number]).columns
payroll[numeric_cols] = payroll[numeric_cols].clip(lower=0)


payroll.fillna(0, inplace=True)
payroll['Office'] = payroll['Office'].astype(int)

# define column for 1/4 year discretization
payroll['Fiscal Quarter'] = payroll['Fiscal Quarter']*0.25 - 0.25
payroll['Fiscal Period'] = payroll['Fiscal Year'] + payroll['Fiscal Quarter']

pd.to_datetime(payroll['Original Hire Date'])

# parse hire date to get hire year
payroll['Original Hire Date'] = payroll['Original Hire Date'].str.split('/').str[2]
payroll['Original Hire Date'] = payroll['Original Hire Date'].astype(int)

_work_year = payroll["Fiscal Period"] - payroll['Original Hire Date']

In [ ]:
# specify table column included

payroll = payroll[['Fiscal Year', 'Fiscal Period', 'Job Code', 'Job Title', 'Base Pay', 'Position ID', 'Employee Identifier']]

# add Working Year Column
payroll["Working Year"] = _work_year

In [ ]:
payroll.isna().sum()

In [ ]:
payroll.info()

In [ ]:
#Melihat heatmap korelasi antarkolom di dataframe Covid
fig, ax = plt.subplots(figsize=(15,8))
sns.heatmap(payroll.corr(), annot=True, fmt='.1g', cmap="Blues", cbar=True, linewidths=0.5, linecolor='grey');

In [ ]:
# pengecekan korelasi dengan death event
corr = payroll.corr()
corr['Base Pay']

In [ ]:
# plot korelasi dengan base pay
sns.heatmap(corr[-1:], annot=False)

In [ ]:
# YANG INI JANGAN DIJALANKAN DULU YA GESSS :'

arr = []

try:
    with open('./cache/ids.txt', "r") as f:
        for _id in f:
            arr.append(int(_id))
except:
    if not os.path.exists('./cache'):
        os.mkdir('./cache')
        
    _index = payroll['Job Code'].unique()
    for _id in _index:
        counts = len(payroll.groupby(['Job Code']).get_group(_id))
        if counts > 1000:
            arr.append(str(_id))
    with open('cache/ids.txt', 'w') as f:
        for _id in arr:
            f.write('%s\n' % _id)
    with open('cache/ids.txt', 'r') as f:
        arr = []
        for _id in f:
            arr.append(int(_id)) 
finally:
   f.close()

# :return : <List> arr : list of unique job id

In [ ]:
emps = []
max = 0;
for _id in arr:
    df = payroll[payroll['Job Code'] == _id]
    emp_id = df['Employee Identifier'].unique()
    if len(emp_id) > 50:
        l = 75
    else:
        l = len(emp_id)

    for i in range(l):
        for i in range(100):
            _index = np.random.randint(0, l-1)
            if emp_id[_index] not in emps:
                emps.append(emp_id[_index])
                break


len(emps)

In [ ]:
df_id = payroll[payroll['Employee Identifier'].isin(emps)]
df_id

In [ ]:
class NodeTree():
    def __init__(cons, attributes=None, threshold=None, left=None, right=None, infoGain=None, value=None):
        
        ''' constructor ''' 
        # Untuk decision node
        cons.attributes = attributes
        cons.threshold = threshold
        cons.left = left
        cons.right = right
        cons.infoGain = infoGain
        
        # Untuk leaf node
        cons.value = value

In [ ]:
class DecisionTreeClassifier():
    def __init__(cons, minSplit=2, maxDepth=3):
        ''' constructor '''
        
        # Inisialisasi root tree
        cons.root = None
        
        # Kondisi untuk menghentikan percabangan
        cons.minSplit = minSplit
        cons.maxDepth = maxDepth
        
    def build_tree(cons, dataset, currentDepth=0):
        ''' fungsi rekursif untuk membangun tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        sizeSamples, sizeAttributes = np.shape(X)
        
        # Melakukan percabangan sampai kondisi tidak memenuhi
        if sizeSamples >= cons.minSplit and currentDepth <= cons.maxDepth:
            # Cari attribute untuk percabangan terbaik
            bestAttribute = cons.find_best_attribute(dataset, sizeSamples, sizeAttributes)
            
            # Cek apakah information gain lebih dari 0, jika iya maka bisa dilanjutkan
            if bestAttribute["infoGain"] > 0:
                # rekursif untuk membentuk left subtree sampai pada leaf
                leftSubtree = cons.build_tree(bestAttribute["datasetLeft"], currentDepth+1)
                
                # rekursif untuk membentuk right subtree sampai pada leaf
                rightSubtree = cons.build_tree(bestAttribute["datasetRight"], currentDepth+1)
                
                # return node decision
                return NodeTree(bestAttribute["attributes"], bestAttribute["threshold"], 
                                leftSubtree, rightSubtree, bestAttribute["infoGain"])
        
        # hitung leaf node
        leafValue = cons.calculate_leaf(Y)
        # return leaf node
        return NodeTree(value=leafValue)
    
    def find_best_attribute(cons, dataset, sizeSamples, sizeAttributes):
        ''' fungsi untuk mencari attribute terbaik '''
        
        # dictionary yang digunakan untuk menyimpan attribute terbaik
        bestAttribute = {}
        maxInfoGain = -float("inf")
        
        # loop untuk melihat keseluruhan attributes
        for attributes in range(sizeAttributes):
            attributeValues = dataset[:, attributes]
            posTreshold = np.unique(attributeValues)
            
            # loop untuk keseluruhan nilai dari attributes di dataset
            for threshold in posTreshold:
                # mengambil percabangan yang terbaru
                datasetLeft, datasetRight = cons.split(dataset, attributes, threshold)
                
                # cek apakah child tree tidak kosong
                if len(datasetLeft)>0 and len(datasetRight)>0:
                    y, left_y, right_y = dataset[:, -1], datasetLeft[:, -1], datasetRight[:, -1]
                    
                    # hitung information gain
                    currentInfoGain = cons.calculate_information_gain(y, left_y, right_y, "gini")
                    # update percabangan terbaik apabila ada
                    if currentInfoGain>maxInfoGain:
                        bestAttribute["attributes"] = attributes
                        bestAttribute["threshold"] = threshold
                        bestAttribute["datasetLeft"] = datasetLeft
                        bestAttribute["datasetRight"] = datasetRight
                        bestAttribute["infoGain"] = currentInfoGain
                        maxInfoGain = currentInfoGain
                        
        # return attribute terbaik
        return bestAttribute
    
    def split(cons, dataset, attributes, threshold):
        ''' fungsi untuk membuat cabang '''
        
        datasetLeft = np.array([row for row in dataset if row[attributes] <= threshold])
        datasetRight = np.array([row for row in dataset if row[attributes] > threshold])
        return datasetLeft, datasetRight
    
    def calculate_information_gain(cons, parent, l_child, r_child, mode="entropy"):
        ''' fungsi untuk menghitung information gain '''
        
        lengthLeft = len(l_child) / len(parent)
        lengthRight = len(r_child) / len(parent)
        if mode=="gini":
            gain = cons.giniValue(parent) - (lengthLeft*cons.giniValue(l_child) + lengthRight*cons.giniValue(r_child))
        else:
            gain = cons.entropy(parent) - (lengthLeft*cons.entropy(l_child) + lengthRight*cons.entropy(r_child))
        return gain
    
    def entropy(cons, y):
        ''' fungsi untuk menghitung entropy '''
        
        labelClass = np.unique(y)
        entropy = 0
        for cls in labelClass:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def giniValue(cons, y):
        ''' fungsi untuk menghitung nilai gini '''
        
        labelClass = np.unique(y)
        gini = 0
        for cls in labelClass:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf(cons, Y):
        ''' fungsi untuk menghitung node leaf '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(cons, tree=None, indent=" "):
        ''' fungsi untuk menampilkan tree yang sudah jadi '''
        
        if not tree:
            tree = cons.root

        if tree.value is not None:
            print(tree.value)

        else:
            print(covid.columns[tree.attributes], "=", tree.threshold, "?", tree.infoGain)
            print("%sbenar: " % (indent), end=" ")
            cons.print_tree(tree.left, indent + indent)
            print("%ssalah: " % (indent), end=" ")
            cons.print_tree(tree.right, indent + indent)
    
    def fit(cons, X, Y):
        ''' fungsi untuk melatih tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        cons.root = cons.build_tree(dataset)
    
    def predict(cons, X):
        ''' fungsi untuk memprediksi dataset '''
        
        predictions = [cons.make_prediction(x, cons.root) for x in X]
        return predictions
    
    def make_prediction(cons, x, tree):
        ''' fungsi untuk memprediksi single data point '''
        
        if tree.value!=None: 
          return tree.value
        feature_val = x[tree.attributes]
        if feature_val<=tree.threshold:
            return cons.make_prediction(x, tree.left)
        else:
            return cons.make_prediction(x, tree.right)

In [ ]:
def confusion_matrixs(pred, original): #pass predicted and original labels to this function
    matrix=np.zeros((2,2)) # form an empty matric of 2x2
    for i in range(len(pred)): #the confusion matrix is for 2 classes: 1,0
        #1=positive, 0=negative
        if int(pred[i])==1 and int(original[i])==1: 
            matrix[0,0]+=1 #True Positives
        elif int(pred[i])==1 and int(original[i])==0:
            matrix[0,1]+=1 #False Positives
        elif int(pred[i])==0 and int(original[i])==1:
            matrix[1,0]+=1 #False Negatives
        elif int(pred[i])==0 and int(original[i])==0:
            matrix[1,1]+=1 #True Negatives
    precision=matrix[0,0]/(matrix[0,0]+matrix[0,1])
    print("Precision:",precision)
    recall=matrix[0,0]/(matrix[0,0]+matrix[1,0])
    print("Recall:",recall)
    specificity=matrix[1,1]/(matrix[0,1]+matrix[1,1])
    print("Specificity:",specificity)
    negative_pred_value=matrix[1,1]/(matrix[1,0]+matrix[1,1])
    print("Negative Predicted Value:",negative_pred_value)
    f2=  2*(precision*recall)/(precision+recall)
    print("F1 score:",f2)

    #the above code adds up the frequencies of the tps,tns,fps,fns and a matrix is formed
    return matrix

def accuracy_metric(pred, original):
  correct = 0
  for i in range(len(original)):
    if original[i] == pred[i]:
      correct += 1
  return correct / float(len(original)) * 100.0

In [ ]:
#Menyiapkan dataset train dan test

X = df_id.iloc[:, :-1].values
Y = df_id.iloc[:, -1].values.reshape(-1,1)

print(X)
print(Y)

In [ ]:
def shuffle_split_data(X, Y):
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, 80)

    X_train = X[split]
    Y_train = Y[split]
    X_test =  X[~split]
    Y_test = Y[~split]

    print(len(X_train), len(Y_train), len(X_test), len(Y_test))
    return X_train, Y_train, X_test, Y_test

try:
    X_train, Y_train, X_test, Y_test = shuffle_split_data(X, Y)
    print("Successful")
except:
    print("Fail")

print('------------')
print(X_train)
print(X_test)
print(Y_train)
print(Y_test)

In [ ]:
classifier = DecisionTreeClassifier(minSplit=3, maxDepth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

In [ ]:
Y_pred = classifier.predict(X_test)

confusion = confusion_matrixs(Y_pred,Y_test)
print('-------------------------------------------------')
print(confusion)
accuracy = accuracy_metric(Y_pred, Y_test)
print('-------------------------------------------------')
print("Test accuracy: {} %".format(accuracy))